In [1]:
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
import shap as shap

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

## read data 
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")

/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## Set up model
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 12, alpha =1.291549665014884, random_state=10, learning_rate = 'adaptive' )
#train data: 2000 - 2010
train_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,2010])]
X_train = train_data[['MSM','MSMW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION']]
y_train = 1 - train_data['Susceptible']

oversample = RandomOverSampler(sampling_strategy = 0.5,random_state=42)
X_train, y_train = oversample.fit_resample(X_train,y_train)
model_fit = model_nn.fit(X_train, y_train)

#test data: 2009 - 2019 
test_data = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])]
X_test = test_data[['MSM','MSMW',  'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION']]
y_test = 1 - test_data['Susceptible']
cipro_R_prev_all= y_test.sum()/len(y_test) #get prevalence overall before radndom oversampling
print(cipro_R_prev_all)
X_test, y_test = oversample.fit_resample(X_test,y_test)

y_predict = model_nn.predict(X_test)


0.23081886682486158


In [5]:
X_test.head()

,MSM,MSMW,Oth/Unk/Missing,Northeast,Southeast,Southwest,West,PREV_REGION,PREV_CLINIC,DELTA_REGION
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092199,0.228571,0.000715
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092199,0.228571,0.000715
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092199,0.228571,0.000715
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092199,0.228571,0.000715
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.092199,0.228571,0.000715


In [7]:
### Get shapley values

explainer = shap.Explainer(model_fit)
shap_values = explainer.shap_values(X_test)

shap.plots.waterfall(shap_values[0])

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: MLPClassifier(activation='tanh', alpha=1.291549665014884, hidden_layer_sizes=12,
              learning_rate='adaptive', max_iter=3000, random_state=10,
              solver='lbfgs')

In [16]:
shap.plots.waterfall(shap_values)

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'